<a href="https://colab.research.google.com/github/janbanot/msc-project/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!uv pip install transformers datasets captum quantus accelerate

In [ ]:
import os
import re
from datetime import datetime
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from torch import nn

In [ ]:
from google.colab import drive
drive.mount('/drive')

## Konfiguracja treningowa

In [ ]:
# ===================================================
# KONFIGURACJA TRENINGOWA
# ===================================================

# === Ścieżki danych i modelu ===
DATA_PATH = "/drive/MyDrive/msc-project/jigsaw-toxic-comment/train.csv"  # Plik CSV z danymi Jigsaw Toxic Comment

# Dodaj timestamp do nazwy katalogu, aby nie nadpisywać poprzednich wyników
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_MODEL_DIR = f"/drive/MyDrive/msc-project/models/distilbert-jigsaw-full_{TIMESTAMP}_FULL_long"  # Katalog zapisu wytrenowanego modelu

# === Parametry modelu ===
MODEL_CHECKPOINT = "distilbert-base-uncased"  # Model bazowy do fine-tuningu
MAX_SEQUENCE_LENGTH = 256  # Maksymalna długość sekwencji tokenów (max dla DistilBERT to 512)

# === Hiperparametry treningu ===
BATCH_SIZE = 16  # Rozmiar batcha treningowego
NUM_EPOCHS = 4  # Liczba epok treningu (standard dla BERT to 2-4 epoki; więcej = ryzyko overfittingu)
LEARNING_RATE = 2e-5  # Learning rate (2e-5 jest standardem dla fine-tuningu BERT; zmiana może destabilizować trening)
WEIGHT_DECAY = 0.01  # Regularyzacja L2 zapobiegająca overfittingowi


## 1. Przygotowanie danych

In [ ]:
# ===================================================
# 1. PRZYGOTOWANIE DANYCH
# ===================================================


def clean_text(example):
    """
    Czyści tekst komentarza, usuwając szum i normalizując format.

    Funkcja stosowana zarówno podczas treningu jak i ewaluacji, aby zapewnić
    spójność przetwarzania danych.

    Argumenty:
        example: Słownik zawierający klucz 'comment_text' z tekstem do oczyszczenia

    Zwraca:
        Zmodyfikowany słownik example z oczyszczonym tekstem

    Operacje czyszczenia:
        - Konwersja na małe litery (wymagane dla uncased BERT)
        - Usunięcie URL (http/https/www)
        - Usunięcie adresów IP
        - Normalizacja białych znaków (zamiana \\n na spacje, collapse wielokrotnych spacji)
    """
    text = example["comment_text"]
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", "", text)
    text = text.replace("\n", " ")
    text = re.sub(r"\s+", " ", text).strip()
    example["comment_text"] = text
    return example


print(">>> Wczytywanie danych...")
df = pd.read_csv(DATA_PATH)

dataset = Dataset.from_pandas(df)
dataset = dataset.map(clean_text)

# === Tokenizacja ===
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)


def tokenize_function(examples):
    """
    Tokenizuje teksty do formatu akceptowanego przez model BERT.

    Argumenty:
        examples: Batch przykładów z kluczem 'comment_text'

    Zwraca:
        Słownik z kluczami: input_ids, attention_mask

    Parametry tokenizacji:
        - padding="max_length": Wyrównuje wszystkie sekwencje do MAX_SEQUENCE_LENGTH
        - truncation=True: Obcina zbyt długie teksty
        - max_length=256: Długość sekwencji (równowaga kontekst/szybkość)
    """
    return tokenizer(
        examples["comment_text"],
        padding="max_length",
        truncation=True,
        max_length=MAX_SEQUENCE_LENGTH,
    )


print(">>> Tokenizacja...")
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# === Przygotowanie etykiet binary classification ===
label_cols = ["toxic"]


def format_labels(example):
    """
    Konwertuje etykietę 'toxic' na format int wymagany przez CrossEntropyLoss.

    Argumenty:
        example: Słownik z kluczem 'toxic' (wartość 0 lub 1)

    Zwraca:
        Zmodyfikowany słownik z dodanym kluczem 'labels' (int)
    """
    example["labels"] = int(example["toxic"])
    return example


tokenized_dataset = tokenized_dataset.map(format_labels)

# Usunięcie zbędnych kolumn (zachowanie tylko danych potrzebnych modelowi)
tokenized_dataset = tokenized_dataset.remove_columns(
    ["id", "comment_text"] + label_cols
)
tokenized_dataset.set_format("torch")

# === Podział na zbiór treningowy i walidacyjny ===
splits = tokenized_dataset.train_test_split(
    test_size=0.2, seed=42
)  # 20% na walidację, 80% na trening
train_ds = splits["train"]
eval_ds = splits["test"]

print(
    f"Dane gotowe. Zbiór treningowy: {len(train_ds)}, Zbiór walidacyjny: {len(eval_ds)}"
)


## 2. Definicja modelu i metryk

In [ ]:
# ===================================================
# 2. DEFINICJA MODELU I METRYK
# ===================================================

# === Załadowanie modelu ===
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=2,  # Zmieniamy na 2 (Toxic vs Non-Toxic)
    id2label={0: "NON_TOXIC", 1: "TOXIC"},
    label2id={"NON_TOXIC": 0, "TOXIC": 1}
)


def compute_metrics(p):
    """
    Oblicza metryki ewaluacyjne modelu: F1, Accuracy, ROC AUC.

    Argumenty:
        p: Krotka (predictions, labels) z predykcjami modelu i etykietami

    Zwraca:
        Słownik z kluczami: f1, accuracy, roc_auc
    """
    predictions, labels = p

    # Dla num_labels=2 wyjście to [batch_size, 2]
    # Wybieramy klasę o wyższym wyniku (argmax na osi 1)
    preds = np.argmax(predictions, axis=1)

    # Do ROC_AUC potrzebujemy prawdopodobieństwa klasy pozytywnej (1)
    # Stosujemy Softmax na logitach
    probs = torch.nn.functional.softmax(torch.tensor(predictions), dim=-1)
    prob_toxic = probs[:, 1].numpy()  # Prawdopodobieństwo klasy 1

    f1 = f1_score(labels, preds)
    acc = accuracy_score(labels, preds)

    try:
        roc_auc = roc_auc_score(labels, prob_toxic)
    except ValueError:
        roc_auc = 0.0

    return {
        "f1": f1,
        "accuracy": acc,
        "roc_auc": roc_auc,
    }


## 3. Konfiguracja i uruchomienie treningu

In [ ]:
# ===================================================
# 3. KONFIGURACJA I URUCHOMIENIE TRENINGU
# ===================================================

training_args = TrainingArguments(
    output_dir=f"{OUTPUT_MODEL_DIR}_checkpoints",  # Katalog dla checkpointów (pośrednich zapisów modelu)
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=WEIGHT_DECAY,  # Regularyzacja L2 zapobiegająca overfittingowi
    eval_strategy="steps",
    save_strategy="steps",  # Zapis checkpointu co save_steps kroków
    logging_steps=100,
    eval_steps=500,
    save_steps=500,
    load_best_model_at_end=True,  # Po treningu załaduj najlepszy model (według metryki metric_for_best_model)
    metric_for_best_model="f1",  # Kryterium wyboru najlepszego modelu (f1 dla binary classification)
    save_total_limit=2,  # Zachowaj tylko 2 ostatnie checkpointy (oszczędność miejsca na dysku)
    fp16=True,  # Mixed precision training (przyspieszenie na GPU; wymaga CUDA)
    report_to="none",  # Wyłączenie integracji z Weights & Biases
)

# 1. Oblicz wagi klas (im więcej zer, tym mniejsza waga dla zera)
# Przybliżone wagi dla Jigsaw (jeśli nie chcesz liczyć dokładnie):
# Klasa 0 (Non-toxic): 1.0
# Klasa 1 (Toxic): ~9.0 (bo jest ich 9x mniej)
class_weights = torch.tensor([1.0, 9.0]).to("cuda" if torch.cuda.is_available() else "cpu")


# 2. Custom Trainer z obsługą wag
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Zabezpieczenie: upewnij się, że wagi są na tym samym urządzeniu co model
        # (w razie gdyby Trainer wewnętrznie przeniósł model na inne GPU)
        weight = class_weights.to(model.device)

        loss_fct = nn.CrossEntropyLoss(weight=weight)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss


# 3. Użycie WeightedTrainer zamiast zwykłego Trainer
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=6)]
)

print(">>> Rozpoczynanie treningu...")
print(
    f"Parametry: {NUM_EPOCHS} epok, batch size {BATCH_SIZE}, learning rate {LEARNING_RATE}"
)
trainer.train()

## 4. Zapis finalnego modelu i wyników

In [ ]:
# ===================================================
# 4. ZAPIS FINALNEGO MODELU I WYNIKÓW
# ===================================================

print(f">>> Zapisywanie modelu do: {OUTPUT_MODEL_DIR}")
trainer.save_model(OUTPUT_MODEL_DIR)  # Zapis modelu (wagi + konfiguracja)
tokenizer.save_pretrained(OUTPUT_MODEL_DIR)  # Zapis tokenizera (słownik + konfiguracja)

# === Ewaluacja finalna na zbiorze walidacyjnym ===
metrics = trainer.evaluate()
print("Metryki finalne:", metrics)

# Zapis metryk do pliku tekstowego (dla dokumentacji)
with open(f"{OUTPUT_MODEL_DIR}/training_results.txt", "w", encoding="utf-8") as f:
    f.write("=== WYNIKI TRENINGU ===\n")
    f.write(str(metrics))

print("\n>>> Trening zakończony pomyślnie!")
print(f"Model zapisany w: {OUTPUT_MODEL_DIR}")


## 5. Analiza progu decyzyjnego

In [ ]:
# ===================================================
# 5. ANALIZA PROGU DECYZYJNEGO
# ===================================================

# === Generowanie predykcji ===
print(">>> Generowanie predykcji dla zbioru walidacyjnego...")
raw_pred, _, _ = trainer.predict(eval_ds)

# Zastosuj Softmax, aby uzyskać prawdopodobieństwa (0-1)
# raw_pred to logity - zamieniamy na prawdopodobieństwo bycia klasą 1 (Toxic)
probs = torch.nn.functional.softmax(torch.tensor(raw_pred), dim=-1)
toxic_probs = probs[:, 1].numpy()  # Prawdopodobieństwo klasy 1
true_labels = np.array(eval_ds["labels"])

# === Szukanie optymalnego progu (Threshold Search) ===
best_f1 = 0
best_thresh = 0

print("\n>>> Szukanie optymalnego progu...")
# Sprawdzamy progi od 0.1 do 0.9 co 0.01
for thresh in np.arange(0.1, 0.91, 0.01):
    # Jeśli prawdopodobieństwo > thresh, to klasa 1, inaczej 0
    preds = (toxic_probs > thresh).astype(int)

    f1 = f1_score(true_labels, preds)

    if f1 > best_f1:
        best_f1 = f1
        best_thresh = thresh

print("--------------------------------------------------")
print(f"Standardowy wynik (argmax / thresh=0.5): F1 ≈ {metrics['eval_f1']:.4f}")
print(f"Najlepszy znaleziony próg:               {best_thresh:.2f}")
print(f"NOWY WYNIK F1 po zmianie progu:          {best_f1:.4f}")
print("--------------------------------------------------")

# === Szczegóły dla najlepszego progu ===
final_preds = (toxic_probs > best_thresh).astype(int)
print(f"Precision: {precision_score(true_labels, final_preds):.4f}")
print(f"Recall:    {recall_score(true_labels, final_preds):.4f}")

## 6. Analiza błędów klasyfikacji

In [ ]:
# ===================================================
# 6. ANALIZA BŁĘDÓW KLASYFIKACJI
# ===================================================

# === Generowanie predykcji ===
raw_pred, _, _ = trainer.predict(eval_ds)
probs = torch.nn.functional.softmax(torch.tensor(raw_pred), dim=-1)
toxic_probs = probs[:, 1].numpy()
true_labels = np.array(eval_ds["labels"])

# === Przygotowanie DataFrame z wynikami ===
# Dekodowanie tokenów z powrotem na tekst
decoded_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in eval_ds["input_ids"]]

results_df = pd.DataFrame({
    "text": decoded_texts,
    "true_label": true_labels,
    "pred_prob": toxic_probs,
    "pred_label": (toxic_probs > 0.5).astype(int)
})

# === False Positives (Model myślał, że toksyczne, a nie było) ===
# Sortuj po pewności modelu (im bliżej 1.0 tym gorzej)
fp = results_df[(results_df["pred_label"] == 1) & (results_df["true_label"] == 0)].sort_values("pred_prob", ascending=False).head(10)

# === False Negatives (Model myślał, że bezpieczne, a było toksyczne) ===
# Sortuj po pewności modelu (im bliżej 0.0 tym gorzej)
fn = results_df[(results_df["pred_label"] == 0) & (results_df["true_label"] == 1)].sort_values("pred_prob", ascending=True).head(10)



In [ ]:
print("=== TOP 5 FALSE POSITIVES (Model widzi toksyczność tam gdzie jej nie ma) ===")
for i, row in fp.head(5).iterrows():
    print(f"Prob: {row['pred_prob']:.4f} | Text: {row['text']}")

print("\n=== TOP 5 FALSE NEGATIVES (Model przegapił toksyczność) ===")
for i, row in fn.head(5).iterrows():
    print(f"Prob: {row['pred_prob']:.4f} | Text: {row['text']}")